# 1. Let's import necessary libraries

In [11]:
import pyemu
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import matplotlib.pyplot as plt
import psutil
import shutil
import numpy as np
import sys
import flopy

## work with the latest version of swatmf

In [2]:
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(1, path)
from swatmf import swatmf_pst_utils

# 2. Set up and write swatmf.con file

In [3]:
swatmf_pst_utils.create_swatmf_con?

Signature:
swatmf_pst_utils.create_swatmf_con(
    wd,
    sim_start,
    warmup,
    cal_start,
    cal_end,
    subs=None,
    grids=None,
    riv_parm=None,
    baseflow=None,
    time_step=None,
    depth_to_water=None,
    pp_included=None,
)
Docstring:
create swatmf.con file containg SWAT-MODFLOW model PEST initial settings

Args:
    wd (`str`): SWAT-MODFLOW working directory
    subs (`list`): reach numbers to be extracted
    grids (`list`): grid numbers to be extracted
    sim_start (`str`): simulation start date e.g. '1/1/2000'
    warmup(`int`): warm-up period
    cal_start (`str`): calibration start date e.g., '1/1/2001'
    cal_end (`str`): calibration end date e.g., '12/31/2005'
    time_step (`str`, optional): model time step. Defaults to None ('day'). e.g., 'day', 'month', 'year'
    riv_parm (`str`, optional): river parameter activation. Defaults to None ('n').
    depth_to_water (`str`, optional): extracting simulated depth to water activation. Defaults to None ('n')

In [4]:
# time step
time_step = 'day'

# extract depth_to_water (what our targets)
depth_to_water = 'y'

# locations
subs = [58]
grids = [501]

# calibration period
sim_start = '1/1/1985'
cal_start = '1/1/1985'
cal_end = '12/31/1985'

In [5]:
# working directory
swatmf_model = "D:/test/middle_bosque_1000/SWAT-MODFLOW"
swat_model = "D:/test/middle_bosque_1000/SWAT"

In [6]:
swatmf_pst_utils.create_swatmf_con(swatmf_model, subs, grids, sim_start, cal_start, cal_end, depth_to_water=depth_to_water)

,names,vals
0,wd,D:/test/middle_bosque_1000/SWAT-MODFLOW
1,sim_start,[58]
2,warm-up,[501]
3,cal_start,1/1/1985
4,cal_end,1/1/1985
5,subs,12/31/1985
6,grids,n
7,riv_parm,n
8,baseflow,n
9,time_step,day


## 2.1 PEST initial setup

In [9]:
# copy all necessary files (exes) to your working direcotry
swatmf_pst_utils.init_setup(swatmf_model, swat_model)

100%|███████████████████████████████████████████████████████████████████████████| 12300/12300 [02:02<00:00, 100.59it/s]

 Creating 'backup' folder ... passed
 Creating 'echo' folder ... passed
 Creating 'sufi2.in' folder ... passed
 'i64pwtadj1.exe' file copied ... passed
 'pestpp-glm.exe' file copied ... passed
 'pestpp-ies.exe' file copied ... passed
 'pestpp-opt.exe' file copied ... passed
 'pestpp-sen.exe' file copied ... passed
 'SUFI2_LH_sample.exe' file copied ... passed
 'Swat_Edit.exe' file copied ... passed


In [13]:
# check MODFLOW model
mname = "modflow.mfn"
m = flopy.modflow.Modflow.load(mname,model_ws=swatmf_model)
m.check()


modflow MODEL DATA VALIDATION SUMMARY:
  8 Errors:
    DIS package: thin cells (less than checker threshold of 1.0)
    RIV package: rbot below cell bottom
  3 Warnings:
    OC package: action(s) defined in OC stress_period_data ignored as they are not part the stress periods defined by DIS
    RCH package: Mean R/T ratio < checker warning threshold of 2e-08 for 1 stress periods
    RCH package: Variable NRCHOP set to value other than 3

  Checks that passed:
    Unit number conflicts
    Compatible solver package
    DIS package: zero or negative thickness
    DIS package: nan values in top array
    DIS package: nan values in bottom array
    BAS6 package: isolated cells in ibound array
    BAS6 package: Not a number
    UPW package: zero or negative horizontal hydraulic conductivity values
    UPW package: zero or negative vertical hydraulic conductivity values
    UPW package: negative horizontal anisotropy values
    UPW package: horizontal hydraulic conductivity values below che

# 3. Build template files

In [14]:
os.chdir(swatmf_model)

## 3.1 MODFLOW pval

In [15]:
# pval file
pval_file = 'mf_1000.pval'

In [16]:
gw_par = pyemu.utils.gw_utils.modflow_pval_to_template_file(pval_file, tpl_file=None)
gw_par

,parnme,parval1,tpl
parnme,,,
hk01,hk01,1.00,~ hk01 ~
hk02,hk02,1.00,~ hk02 ~
sy01,sy01,0.01,~ sy01 ~
sy02,sy02,0.01,~ sy02 ~


## 3.2 SWAT model.in file

In [17]:
# model.in file used

sw_par = swatmf_pst_utils.model_in_to_template_file()
sw_par

,parnme,parval1,tpl
parnme,,,
r__CN2.mgt,r__CN2.mgt,0.001,~ CN2 ~
r__SOL_AWC().sol,r__SOL_AWC().sol,0.001,~ SOL_AWC() ~
v__ESCO.hru,v__ESCO.hru,0.001,~ ESCO ~


# 4. Build instruction files

### Let's do initial run!

## 4.1 Streamflow (SWAT)

In [ ]:
# extract daily stream discharge
swatmf_pst_utils.extract_day_stf(subs, sim_start, cal_start, cal_end)

## 4.2 match it with stf_obd file (SWAT)

In [ ]:
swatmf_pst_utils.stf_obd_to_ins('str_058.txt', 'rch058',cal_start, cal_end)

## 4.3 Depth to watertable (MODFLOW) 

In [ ]:
swatmf_pst_utils.extract_depth_to_water(grids, sim_start, cal_end)

## 4.4 match it with dtw_obd file (MODFLOW)

In [ ]:
swatmf_pst_utils.mf_obd_to_ins('dtw_501.txt', 'g_5699', cal_start, cal_end)

# 5. Create PEST control file

In [ ]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)
pyemu.helpers.pst_from_io_files(io_files[0], io_files[1], io_files[2], io_files[3], 'mb_dummy.pst')

In [ ]:
par = pst.parameter_data
par

## 5.1 Assign parameter group name

In [ ]:
for i in range(len(par)):
    if (par.iloc[i, 0][:2]) == 'sy':
        par.iloc[i, 6] = 'sy'
    elif par.iloc[i, 0][:7] == 'rivbot_':
        par.iloc[i, 6] = 'rivbot'
    elif par.iloc[i, 0][:6] == 'rivcd_':
        par.iloc[i, 6] = 'rivcd'
    elif par.iloc[i, 0][:2] == 'hk':
        par.iloc[i, 6] = 'hk'
    else:
        par.iloc[i, 6] = 'swat'
print(par)

## 5.2 Adjust initial parameter values and their ranges

In [ ]:
count = 0
for i in range(len(par)):
    if (par.iloc[i, 6] == 'hk'):
        par.iloc[i, 3] = 1  
        par.iloc[i, 4] = 1.000000e-02
        par.iloc[i, 5] = 1.000000e+02
    elif (par.iloc[i, 6] == 'sy'):
        par.iloc[i, 3] = 1.000000e-02       
        par.iloc[i, 4] = 1.000000e-04
        par.iloc[i, 5] = 0.6  
    elif (par.iloc[i, 6] == 'rivbot'):
        par.iloc[i, 3] = 3.001     
        par.iloc[i, 4] = 0.001
        par.iloc[i, 5] = 6
        par.iloc[i, 8] = -3
    elif (par.iloc[i, 6] == 'rivcd'):
        par.iloc[i, 3] = 50.001       
        par.iloc[i, 4] = 0.001
        par.iloc[i, 5] = 100
        par.iloc[i, 8] = -50
    else:
        count += 1
count

In [ ]:
# CN2
par.loc['cn2', 'parval1'] = 1.001
par.loc['cn2', 'parlbnd'] = 0.8
par.loc['cn2', 'parubnd'] = 1.2
par.loc['cn2', 'offset'] = -1

# ESCO
par.loc['esco', 'parval1'] = 1.001
par.loc['esco', 'parlbnd'] = 0.5
par.loc['esco', 'parubnd'] = 1.5
par.loc['esco', 'offset'] = -1

# sol_awc()
par.loc['sol_awc()', 'parval1'] = 1.001
par.loc['sol_awc()', 'parlbnd'] = 0.5
par.loc['sol_awc()', 'parubnd'] = 1.5
par.loc['sol_awc()', 'offset'] = -1


## 5.3 Assign parameter group name

In [ ]:
# set observation group
obd = pst.observation_data
obd

In [ ]:
# Change obd group name
for i in range(len(obd)):
    obd.iloc[i, 3] = obd.iloc[i, 0][:-9]
obd

## 5.4 Provide actual observed values to control file

In [ ]:
# Streamflow
stf_obd = pd.read_csv('stf_day.obd',
                       sep='/t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
stf_obd = stf_obd[cal_start: cal_end]
stf_obd

In [ ]:
# watertable
dtw_obd = pd.read_csv('dtw_day.obd',
                       sep='/t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
dtw_obd = dtw_obd[cal_start: cal_end]
dtw_obd

In [ ]:
# Get sub list based on obd order
obd_order = []
for i in obd.obgnme.tolist():
    if i not in obd_order:
        obd_order.append(i)
obd_order

In [ ]:
# get total list from each sub obd, delete na vals
tot_obd = []
for i in obd_order[:1]:
    tot_obd += dtw_obd[i].dropna().tolist()
    print(i)
for i in obd_order[1:]:
    tot_obd += stf_obd[i].dropna().tolist()
    print(i)
len(tot_obd)

In [ ]:
obd.loc[:, 'obsval'] = tot_obd
obd

# 6. Create new control file with settings

In [ ]:
pst.control_data.noptmax=0
pst.model_command = 'python forward_run.py'

In [ ]:
pst.write('mb_pest.pst')